In [ ]:
from autogen_ext.models.openai import (
    AzureOpenAIChatCompletionClient,
    OpenAIChatCompletionClient,
)
import os
import dotenv
from autogen_ext.models.openai.config import ResponseFormat
from logging import FileHandler, Formatter, Logger, getLogger, INFO
from datetime import datetime
from colorama import Fore, Style

dotenv.load_dotenv()


def clear_console():
    os.system("cls")


def print_answer(answer):
    print(Style.BRIGHT + Fore.GREEN + f"Final answer:\n{answer}" + Style.RESET_ALL)


def get_azure_model_client():
    model_client = AzureOpenAIChatCompletionClient(
        model="gpt-4o",
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_endpoint=os.getenv("AZURE_OPENAI_API_ENDPOINT"),
        api_version="2024-08-01-preview",
        temperature=0,
    )
    return model_client


def get_openai_model_client():
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o", api_key=os.getenv("OPENAI_API_KEY"), temperature=0
    )
    return model_client


def get_openai_json_model_client():
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o",
        api_key=os.getenv("OPENAI_API_KEY"),
        temperature=0,
        response_format=ResponseFormat({"type": "json_object"}),
    )
    return model_client


def get_logger(name: str) -> Logger:
    log_filename = datetime.now().strftime("%Y%m%d_%H%M%S.log")
    dir_path = os.path.join("fhl_logs", name)
    file_path = os.path.join(dir_path, log_filename)
    os.makedirs(dir_path, exist_ok=True)

    file_handler = FileHandler(file_path)
    formatter = Formatter("%(asctime)s:\n%(message)s\n\n")
    file_handler.setLevel(INFO)
    file_handler.setFormatter(formatter)
    logger = getLogger(name)
    logger.addHandler(file_handler)
    logger.setLevel(INFO)

    return logger


In [ ]:
import asyncio
import json
from autogen_ext.tools.langchain import LangChainToolAdapter
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from langchain_community.tools.arxiv import ArxivQueryRun
from datetime import datetime

logger = get_logger("reflection")

async def main() -> None:
    model_client = get_openai_model_client()

    tool = LangChainToolAdapter(ArxivQueryRun())

    assistant = AssistantAgent(
        "Researcher",
        tools=[tool],
        model_client=model_client,
        system_message=f"Act as a research assistant. Find recent research papers on a given topic and prepare a report. Current date: {datetime.now().strftime("%Y-%m-%d")}.",
        description="An agent that finds recent research papers on a given topic and prepares a report.",
    )

    validator = AssistantAgent(
        "Reviewer",
        model_client=model_client,
        system_message=f"Act as a scientific reviewer and editor. Mark any inconsistencies you can find. Current date: {datetime.now().strftime("%Y-%m-%d")}.",
        description="An agent that reviews the articles found by the researcher.",
    )

    team = MagenticOneGroupChat(
        [assistant, validator], model_client=model_client, max_turns=5, max_stalls=2
    )

    result = await team.run(
        task=f"""Prepare a list of three recent articles in the field of psychology that would be interesting for software engineers.
 Ensure that the list meets reviewer's criteria.
 Current date: {datetime.now().strftime("%Y-%m-%d")}.""",
    )

    answer = result.messages[-1].content

    print_answer(answer)

    state = await team.save_state()

    logger.info(answer)
    logger.info(json.dumps(state, indent=2))


asyncio.run(main())


In [ ]:
import asyncio
import json
from autogen_ext.tools.langchain import LangChainToolAdapter
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from langchain_community.tools.arxiv import ArxivQueryRun
from utils import clear_console, get_logger, get_openai_model_client, print_answer
from datetime import datetime

logger = get_logger("reflection")

clear_console()


async def main() -> None:
    model_client = get_openai_model_client()

    tool = LangChainToolAdapter(ArxivQueryRun())

    assistant = AssistantAgent(
        "Researcher",
        tools=[tool],
        model_client=model_client,
        system_message=f"Act as a research assistant. Find recent research papers on a given topic and prepare a report. Current date: {datetime.now().strftime("%Y-%m-%d")}.",
        description="An agent that finds recent research papers on a given topic and prepares a report.",
    )

    validator = AssistantAgent(
        "Reviewer",
        model_client=model_client,
        system_message=f"Act as a scientific reviewer and editor. Mark any inconsistencies you can find.  Current date: {datetime.now().strftime("%Y-%m-%d")}.",
        description="An agent that reviews the articles found by the researcher.",
    )

    team = MagenticOneGroupChat(
        [assistant, validator], model_client=model_client, max_turns=5, max_stalls=2
    )

    result = await team.run(
        task=f"""Prepare a list of three recent articles in the field of psychology that would be interesting for software engineers.
 Ensure that the list meets reviewer's criteria.
 Current date: {datetime.now().strftime("%Y-%m-%d")}.""",
    )

    answer = result.messages[-1].content

    print_answer(answer)

    state = await team.save_state()

    logger.info(answer)
    logger.info(json.dumps(state, indent=2))


asyncio.run(main())
